### To Do:

- Add images to component drop down
- Attach "incorrect" messages
- Fix image embedding (include image folder with notebook)

# Resizer in Programmable Logic (PL)

This reference design illustrates how to run a resizer IP on the **Programmable Logic (PL)** using Jupyter Notebooks and Python.

<img src="./images/resizenb.png" width="800" height="650"/>

In this example, we will use the **Bilinear** interpolation.

> The resize IP transforms the source image to the size of the destination image. Different types of interpolation techniques can be used in resize function,
namely: Nearest-neighbor, Bilinear, and Area interpolation. The type of interpolation can be passed as a parameter to the Python API.    
> Reference: https://www.xilinx.com/support/documentation/sw_manuals/xilinx2017_1/ug1233-xilinx-opencv-user-guide.pdf

## Contents    
  

* [Image Resizing in Programmable Logic](#Image-Resizing-in-Programmable-Logic)
* [Import libraries](#Import-libraries)
* [Download the Resize IP bitstream](#Download-the-Resize-IP-bitstream)
* [Create an Image object using PIL in SW](#Create-an-Image-object-using-PIL-in-SW)
* [Display the image to be resized](#Display-the-image-to-be-resized)
* [Resizing](#Resizing)
* [Display resized image](#Display-resized-image)
* [References](#References)   




In [1]:
from lib.dropdown import *
from lib.cust_widgets_2 import *
from pynq import Overlay

## Import libraries


In [2]:
#from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from pynq import Xlnk, Overlay

## Image Resizing in Programmable Logic

<img src="attachment:resize_bd.png" style="max-width:100%">


In [3]:
#Pull up a dropdown menu to see more information about the components of this block diagram. 
flowdd()

Flow Components: 

interactive(children=(Dropdown(description=' ', options=(('PS', 0), ('PL', 1), ('ARM', 2), ('DRAM', 3), ('AXI …

## Download the Resize IP bitstream

After the bitstream has been downloaded, the PL will be populated with
the resize IP, the DMA engine, and a few other components. The resize IP
is configured to use bilinear interpolation.

In [4]:
resize_design = Overlay("resizer.bit")


OSError: Bitstream file resizer.bit does not exist.

Then create DMA and Resizer IP objects.

In [5]:
dma = resize_design.axi_dma_0
resizer = resize_design.resize_accel_0

NameError: name 'resize_design' is not defined


<img src="./images/resize_bit.png" width="600" height="450"/>


## Create an Image object using PIL in SW
We will load image from the SD card and create a PIL Image object.

Image By Aboubakrhadnine - <span class="int-own-work" lang="en">Own work</span>, <a href="https://creativecommons.org/licenses/by-sa/4.0" title="Creative Commons Attribution-Share Alike 4.0">CC BY-SA 4.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=79587474">Link</a>.

In [6]:
image_path = "images/sahara.jpg"
original_image = Image.open(image_path)

AttributeError: 'str' object has no attribute 'comm'

<img src="images/sd2dram.png" width="650" height="550"/>

## Display the image to be resized

Let's also check the original image size. It may take a while to render a 
large picture. For better visual effect, we double the size of the canvas.
The following code only changes the display size, not the picture itself:

```python
canvas = plt.gcf()
size = canvas.get_size_inches()
canvas.set_size_inches(size*2)
```

In [7]:
canvas = plt.gcf()
size = canvas.get_size_inches()
canvas.set_size_inches(size*2)

old_width, old_height = original_image.size
print("Image size: {}x{} pixels.".format(old_width, old_height))
_ = plt.imshow(original_image)

NameError: name 'original_image' is not defined

## Resizing
 
We can set resize dimensions.
1. Downscale factor: 2.
2. Size of input image is 3840x2160.
3. Size of output image is 1920x1080.

In [8]:
resize_factor = 2
new_width = int(old_width/resize_factor)
new_height = int(old_height/resize_factor)

NameError: name 'old_width' is not defined

We now allocate memory to process data on PL.
Data is provided as contiguous memory blocks.
The size of the buffer depends on the size of the input or output data.
The image dimensions extracted from the read image are used to allocate contiguous memory blocks.   
We will call `cma_array()` to perform the allocation.

In [9]:
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(old_height, old_width, 3), 
                           dtype=np.uint8, cacheable=1)
out_buffer = xlnk.cma_array(shape=(new_height, new_width, 3), 
                            dtype=np.uint8, cacheable=1)

NameError: name 'Xlnk' is not defined

Documentation snippet for `xlnk.cma_array`:

```python
"""
Get a contiguously allocated numpy array

Parameters
----------
shape : int or tuple of int
    The dimensions of the array to construct - We use (height, width, depth)
dtype : numpy.dtype or str
    The data type to construct - We use 8-bit unsigned int

"""
```

Note that the original image has to be copied into the contiguous memory array (deep copy).

<img src="./images/local2shared.png" width="650" height="550"/>

In [10]:
in_buffer[:] = np.array(original_image)

NameError: name 'original_image' is not defined

We can now run the resizer IP.
We will push the data from input buffer through the pipeline to the output buffer.
For ease-of-use, we will define a `run_kernel` function that will be called 
multiple times.

<img src="./images/resize_accel.png" width="650" height="550"/>

In [11]:
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)    
    resizer.write(0x00,0x81) # start
    dma.sendchannel.wait()
    dma.recvchannel.wait()

We will also need to setup resizer and DMA IPs using MMIO interface 
before we stream image data to them.
For example, we need to write dimensions data to MMIO registers of resizer.
These register writings only have to be done once.

| register offset | configuration                          |
|-----------------|----------------------------------------|
| 0x10            | number of rows for original picture    |
| 0x18            | number of columns for original picture |
| 0x20            | number of rows for resized picture     |
| 0x28            | number of columns for resized picture  |

In [12]:
resizer.write(0x10, old_height)
resizer.write(0x18, old_width)
resizer.write(0x20, new_height)
resizer.write(0x28, new_width)

NameError: name 'resizer' is not defined

Now we can perform the resizing operation.

In [13]:
run_kernel()
resized_image = Image.fromarray(out_buffer)

AttributeError: 'str' object has no attribute 'sendchannel'

## Display resized image

In [14]:
print("Image size: {}x{} pixels.".format(new_width, new_height))
_ = plt.imshow(resized_image)

NameError: name 'new_width' is not defined

We can time the resize in PL operation.

In [15]:
%%timeit
run_kernel()
resized_image = Image.fromarray(out_buffer)

AttributeError: 'str' object has no attribute 'sendchannel'

Finally we need to reset all the contiguous memory buffers.

In [16]:
xlnk.xlnk_reset()

NameError: name 'xlnk' is not defined

### Assessment Questions:

In [17]:
qs=questions_list()

In [18]:
display(qs[0])

Incorrect; think about how the data is pre-processed, and how the data flows between main components.

Incorrect; recall the purpose of the connecting components. 

In [19]:
display(qs[1])

Incorrect; think about which components are needed for the DRAM and the DMA to interact. 

[Contents](#Contents)

# References

https://pillow.readthedocs.io/en/3.1.x/index.html   
https://github.com/Xilinx/PYNQ/blob/master/docs/source/python_environment.ipynb   
https://github.com/Xilinx/PYNQ/blob/master/docs/source/jupyter_notebooks.ipynb   
https://github.com/Xilinx/PYNQ/blob/master/docs/source/jupyter_notebooks_advanced_features.ipynb   